In [1]:
import gc

def free_gpu():
  torch.cuda.empty_cache()
  gc.collect()

In [2]:
path_pp = "/content/gdrive/MyDrive/2022_Research/Machine_Translation/Dataset/Pre_processing/"
path_train = path_pp + "train.csv"
path_test = path_pp + "test.csv"

In [3]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
from torch import cuda
device = 'cuda'if cuda.is_available() else 'cpu'
!nvidia-smi

Tue Jun 27 14:09:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    11W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
import torch, os
import numpy as np
import random

seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
random.seed(seed)
np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [6]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import pandas as pd

df_train = pd.read_csv(path_train)
df_train.head(10)

,vi,zh,vi_token,zh_token,vi_punc,zh_punc
0,Tôi vừa có kế hoạch thay họ biểu diễn ở bữa tiệc.,正打算让这群废物在派对上表演呢,▁tôi ▁vừa ▁có ▁kế ▁hoạch ▁thay ▁họ ▁biểu ▁diễn...,▁正 打算 让 这 群 废 物 在 派 对 上 表演 呢,Tôi vừa có kế hoạch thay họ biểu diễn ở bữa tiệc.,正打算让这群废物在派对上表演呢
1,Không! Cậu đã lấy đi tất cả những gì quý giá đ...,我有价值的东西早被你榨取光了 托雷托先生,▁không ! ▁cậu ▁đã ▁lấy ▁đi ▁tất ▁cả ▁những ▁gì...,▁我 有 价值 的东西 早 被 你 榨 取 光 了 ▁托 雷 托 先生,Không! Cậu đã lấy đi tất cả những gì quý giá đ...,我有价值的东西早被你榨取光了 托雷托先生
2,Tôi luôn đi đến những quán bar đông đúc trong ...,我总是要去拥挤的酒吧，当我真的宁愿得到一个好的和朋友一起吃饭。,▁tôi ▁luôn ▁đi ▁đến ▁những ▁quán ▁bar ▁đông ▁đ...,"▁我 总 是要 去 拥 挤 的 酒吧 , 当 我 真的 宁 愿 得到 一个 好的 和 朋友 ...",Tôi luôn đi đến những quán bar đông đúc trong ...,我总是要去拥挤的酒吧，当我真的宁愿得到一个好的和朋友一起吃饭。
3,"Dương Chấn là người Hoa Âm thời Đông Hán, ông ...",杨震是东汉时华阴人，以教授学生为生，由于正直、博学，人称“关西孔子”。,▁dương ▁chấn ▁là ▁người ▁hoa ▁âm ▁thời ▁đông ▁...,"▁杨 震 是 东汉 时 华 阴 人 , 以 教授 学生 为生 , 由于 正直 、 博 学 ,...","Dương Chấn là người Hoa Âm thời Đông Hán, ông ...","杨震是东汉时华阴人，以教授学生为生，由于正直、博学，人称""关西孔子""。"
4,"Và thực tế là, nếu tôi biết và đồng nghiệp tôi...",实际上，如果我知道和我的同事知道我的一个骨科同事在我的医院，相信我，我和APOS很难与那个人...,"▁và ▁thực ▁tế ▁là , ▁nếu ▁tôi ▁biết ▁và ▁đồng ...","▁ 实际上 , 如果 我 知道 和 我的 同事 知道 我 的一个 骨 科 同事 在 我的 医...","Và thực tế là, nếu tôi biết và đồng nghiệp tôi...",实际上，如果我知道和我的同事知道我的一个骨科同事在我的医院，相信我，我和APOS很难与那个人...
5,"Watt, ghi vào sổ cái. ""Tấn công tàu địch lúc 6...","记在船长日志 ""第六声钟响时迎敌""","▁watt , ▁ghi ▁vào ▁sổ ▁cái . ▁"" tấn ▁công ▁tàu...","▁ 记 在 船长 日 志 ▁"" 第六 声 钟 响 时 迎 敌 ""","Watt, ghi vào sổ cái. ""Tấn công tàu địch lúc 6...","记在船长日志 ""第六声钟响时迎敌"""
6,Khi anh có thể nói sự thật thì mọi việc sẽ dễ ...,能说真话 事情会容易很多,▁khi ▁anh ▁có ▁thể ▁nói ▁sự ▁thật ▁thì ▁mọi ▁v...,▁能 说 真 话 ▁ 事情 会 容易 很多,Khi anh có thể nói sự thật thì mọi việc sẽ dễ ...,能说真话 事情会容易很多
7,4 thầy trò chúng ta lần này đi thỉnh kinh có t...,我们师徒四人这次取西经可以说是困难重重,▁0 ▁thầy ▁trò ▁chúng ▁ta ▁lần ▁này ▁đi ▁thỉnh ...,▁ 我们 师 徒 四人 这次 取 西 经 可以 说是 困难 重重,4 thầy trò chúng ta lần này đi thỉnh kinh có t...,我们师徒四人这次取西经可以说是困难重重
8,Vậy đi tìm chỗ ông ấy ăn trưa và báo lại cho rôi.,那就查出他去了哪儿然后告诉我,▁vậy ▁đi ▁tìm ▁chỗ ▁ông ▁ấy ▁ăn ▁trưa ▁và ▁báo...,▁那 就 查出 他 去了 哪 儿 然后 告诉 我,Vậy đi tìm chỗ ông ấy ăn trưa và báo lại cho rôi.,那就查出他去了哪儿然后告诉我
9,trên gác vọng xuống tiếng bước chân lộn xộn,楼上传来凌乱的脚步声。,▁trên ▁gác ▁vọng ▁xuống ▁tiếng ▁bước ▁chân ▁lộ...,▁ 楼 上传 来 凌 乱 的 脚 步 声 。,trên gác vọng xuống tiếng bước chân lộn xộn,楼上传来凌乱的脚步声。


In [8]:
columns_to_drop = ['vi', 'zh', "vi_punc", "zh_punc"]
df_train = df_train.drop(columns=columns_to_drop)
df_train = df_train.rename(columns = {
                                        'vi_token': 'vi',
                                        'zh_token': 'zh'
                                    })

df_train

,vi,zh
0,▁tôi ▁vừa ▁có ▁kế ▁hoạch ▁thay ▁họ ▁biểu ▁diễn...,▁正 打算 让 这 群 废 物 在 派 对 上 表演 呢
1,▁không ! ▁cậu ▁đã ▁lấy ▁đi ▁tất ▁cả ▁những ▁gì...,▁我 有 价值 的东西 早 被 你 榨 取 光 了 ▁托 雷 托 先生
2,▁tôi ▁luôn ▁đi ▁đến ▁những ▁quán ▁bar ▁đông ▁đ...,"▁我 总 是要 去 拥 挤 的 酒吧 , 当 我 真的 宁 愿 得到 一个 好的 和 朋友 ..."
3,▁dương ▁chấn ▁là ▁người ▁hoa ▁âm ▁thời ▁đông ▁...,"▁杨 震 是 东汉 时 华 阴 人 , 以 教授 学生 为生 , 由于 正直 、 博 学 ,..."
4,"▁và ▁thực ▁tế ▁là , ▁nếu ▁tôi ▁biết ▁và ▁đồng ...","▁ 实际上 , 如果 我 知道 和 我的 同事 知道 我 的一个 骨 科 同事 在 我的 医..."
...,...,...
300343,"▁ông ▁nguyễn ▁đình ▁phúc , ▁người ▁làng ▁phù ▁...","▁ 芙 留 村 居民 阮 庭 福 表示 :"" 过去 , 芙 留 村民 主要 做 买卖 , 仅..."
300344,▁bởi ▁thế ▁làng ▁phù ▁lưu ▁có ▁cái ▁tục ▁rất ▁...,"▁因此 , 我 村 有 认 干 儿 女的 习俗 。"
300345,"▁con ▁cái ▁thường ▁nhờ ▁người ▁khác ▁nuôi , ▁c...","▁ 自己的 孩子 常 托 别人 照 管 、 养 育 , 父母 则 奔 波 各地 谋 生 。"
300346,▁một ▁thời ▁làng ▁phù ▁lưu ▁là ▁làng ▁có ▁nhiề...,"▁我 村 曾是 北部 家里 儿女 认 干 亲 最多的 村庄 ""。"


In [9]:
df_test = pd.read_csv(path_test)
df_test.head(10)

,vi,zh,vi_token,zh_token,vi_punc,zh_punc
0,Cô Dung đã thiết kế và dành ra hai ngày để viế...,蓉女士设计并花了两天时间为班上的 37 名学生撰写评论,▁cô ▁dung ▁đã ▁thiết ▁kế ▁và ▁dành ▁ra ▁hai ▁n...,▁ 蓉 女士 设计 并 花了 两天 时间为 班 上的 ▁00 ▁名 学生 撰写 评论,Cô Dung đã thiết kế và dành ra hai ngày để viế...,蓉女士设计并花了两天时间为班上的 37 名学生撰写评论
1,Nhân dân Triều Tiên thường yêu mến gọi ông là ...,朝鲜人喜欢叫他白马将军，为什么呢？,▁nhân ▁dân ▁triều ▁tiên ▁thường ▁yêu ▁mến ▁gọi...,"▁朝鲜 人 喜欢 叫他 白马 将军 , 为什么 呢 ?",Nhân dân Triều Tiên thường yêu mến gọi ông là ...,朝鲜人喜欢叫他白马将军，为什么呢？
2,Mỗi năm tôi chọn bốn sinh viên đến làm việc ch...,每年 我会挑选4名学生为我工作,▁mỗi ▁năm ▁tôi ▁chọn ▁bốn ▁sinh ▁viên ▁đến ▁là...,▁ 每年 ▁我 会 挑选 0 名学生 为 我 工作,Mỗi năm tôi chọn bốn sinh viên đến làm việc ch...,每年 我会挑选4名学生为我工作
3,"Theo chuyên gia Wyckoff, nhà đầu tư ban đầu ...",根据威科夫的说法，投资者最初对围绕 COVID-19 疫苗的消息感到兴奋。,"▁theo ▁chuyên ▁gia ▁wy ck off , ▁nhà ▁đ â ̀ u ...","▁根据 威 科夫 的说法 , 投资者 最初 对 围绕 ▁c ov id -00 ▁ 疫苗 的...","Theo chuyên gia Wyckoff, nhà đầu tư ban đầu ...",根据威科夫的说法，投资者最初对围绕 COVID-19 疫苗的消息感到兴奋。
4,Nhân viên ngân hàng kỹ năng siêu trộm: Móc 6 t...,银行员工超级小偷技巧：从ATM上钩60亿太容易了,▁nhân ▁viên ▁ngân ▁hàng ▁kỹ ▁năng ▁siêu ▁trộm ...,▁ 银行 员工 超级 小 偷 技巧 : 从 at m 上 钩 00 亿 太 容易 了,Nhân viên ngân hàng kỹ năng siêu trộm: Móc 6 t...,银行员工超级小偷技巧：从ATM上钩60亿太容易了
5,Họ nói rằng họ không có đủ bằng chứng để truy tố.,他们说，他们没有 足够的证据起诉。,▁họ ▁nói ▁rằng ▁họ ▁không ▁có ▁đủ ▁bằng ▁chứng...,"▁ 他们 说 , 他们 没有 ▁ 足够的 证据 起诉 。",Họ nói rằng họ không có đủ bằng chứng để truy tố.,他们说，他们没有 足够的证据起诉。
6,"Không, không, đợi đã, chúng ta không thật sự c...",不，不，等一下 我们不是真的要分手，对么？,"▁không , ▁không , ▁đợi ▁đã , ▁chúng ▁ta ▁không...","▁不 , 不 , 等 一下 ▁ 我们 不是 真的 要 分手 , 对 么 ?","Không, không, đợi đã, chúng ta không thật sự c...",不，不，等一下 我们不是真的要分手，对么？
7,UAE đã trở thành thị trường xuất khẩu tiềm năn...,最近UAE已成为了越南出口商品的潜力市场。,▁uae ▁đã ▁trở ▁thành ▁thị ▁trường ▁xuất ▁khẩu ...,▁ 最近 u ae 已 成为了 越南 出口 商品 的 潜力 市场 。,UAE đã trở thành thị trường xuất khẩu tiềm năn...,最近UAE已成为了越南出口商品的潜力市场。
8,"Theo số liệu của Bộ Thương mại Campuchia, tron...",据柬埔寨贸易部的数据显示，2011年头8个月柬埔寨对越南的商品出口金额达1.052亿美元，较...,▁theo ▁số ▁liệu ▁của ▁bộ ▁thương ▁mại ▁campuch...,"▁据 柬埔寨 贸易 部的 数据 显示 ,0000 年 头 0 个月 柬埔寨 对 越南的 商品...","Theo số liệu của Bộ Thương mại Campuchia, tron...",据柬埔寨贸易部的数据显示，2011年头8个月柬埔寨对越南的商品出口金额达1.052亿美元，较...
9,Bác có nghĩ rằng những người đó sẽ tấn công ch...,你觉得那些人会攻击我们？,▁bác ▁có ▁nghĩ ▁rằng ▁những ▁người ▁đó ▁sẽ ▁tấ...,▁你 觉得 那些 人 会 攻击 我们 ?,Bác có nghĩ rằng những người đó sẽ tấn công ch...,你觉得那些人会攻击我们？


In [10]:
columns_to_drop = ['vi', 'zh', "vi_punc", "zh_punc"]
df_test = df_test.drop(columns=columns_to_drop)
df_test = df_test.rename(columns = {
                                        'vi_token': 'vi',
                                        'zh_token': 'zh'
                                    })

df_test

,vi,zh
0,▁cô ▁dung ▁đã ▁thiết ▁kế ▁và ▁dành ▁ra ▁hai ▁n...,▁ 蓉 女士 设计 并 花了 两天 时间为 班 上的 ▁00 ▁名 学生 撰写 评论
1,▁nhân ▁dân ▁triều ▁tiên ▁thường ▁yêu ▁mến ▁gọi...,"▁朝鲜 人 喜欢 叫他 白马 将军 , 为什么 呢 ?"
2,▁mỗi ▁năm ▁tôi ▁chọn ▁bốn ▁sinh ▁viên ▁đến ▁là...,▁ 每年 ▁我 会 挑选 0 名学生 为 我 工作
3,"▁theo ▁chuyên ▁gia ▁wy ck off , ▁nhà ▁đ â ̀ u ...","▁根据 威 科夫 的说法 , 投资者 最初 对 围绕 ▁c ov id -00 ▁ 疫苗 的..."
4,▁nhân ▁viên ▁ngân ▁hàng ▁kỹ ▁năng ▁siêu ▁trộm ...,▁ 银行 员工 超级 小 偷 技巧 : 从 at m 上 钩 00 亿 太 容易 了
...,...,...
995,▁tôi ▁không ▁thể ▁bảo ▁vệ ▁anh . ▁vậy ▁nên ▁tô...,▁我 不能 保护 你 ▁ 所以 我要 你 离开 这里
996,"▁lượng ▁rau ▁củ ▁quả , ▁trái ▁cây ▁trung ▁quốc...","▁中国 蔬菜 和 水果 在 市场 上的 销售 量 仍然 不少 , 但 数量 已 大幅 下降 。"
997,▁ngành ▁khách ▁sạn ▁khó ▁thoát ▁khỏi ▁vòng ▁la...,"▁ 酒店 业 难 跳 出 周期 ▁截至 ▁0 ▁月 , 胡志明市 旅游 局 报告 称 , 约..."
998,"▁nhiều ▁dự ▁báo ▁cho ▁rằng , ▁vàng ▁sẽ ▁còn ▁t...","▁许多 预测 称 , 在中 美 冲突 高 涨 、 美国 大选 临近 、 新 冠 疫情 仍 不..."


In [11]:
def remove_punc(text):
    return text.replace("▁", "")

In [12]:
df_train["vi"] = df_train["vi"].apply(remove_punc)
df_train["zh"] = df_train["zh"].apply(remove_punc)
df_train

,vi,zh
0,tôi vừa có kế hoạch thay họ biểu diễn ở bữa ti...,正 打算 让 这 群 废 物 在 派 对 上 表演 呢
1,không ! cậu đã lấy đi tất cả những gì quý giá ...,我 有 价值 的东西 早 被 你 榨 取 光 了 托 雷 托 先生
2,tôi luôn đi đến những quán bar đông đúc trong ...,"我 总 是要 去 拥 挤 的 酒吧 , 当 我 真的 宁 愿 得到 一个 好的 和 朋友 一..."
3,"dương chấn là người hoa âm thời đông hán , ông...","杨 震 是 东汉 时 华 阴 人 , 以 教授 学生 为生 , 由于 正直 、 博 学 , ..."
4,"và thực tế là , nếu tôi biết và đồng nghiệp tô...","实际上 , 如果 我 知道 和 我的 同事 知道 我 的一个 骨 科 同事 在 我的 医院..."
...,...,...
300343,"ông nguyễn đình phúc , người làng phù lưu , ch...","芙 留 村 居民 阮 庭 福 表示 :"" 过去 , 芙 留 村民 主要 做 买卖 , 仅有..."
300344,bởi thế làng phù lưu có cái tục rất hay là cho...,"因此 , 我 村 有 认 干 儿 女的 习俗 。"
300345,"con cái thường nhờ người khác nuôi , còn các c...","自己的 孩子 常 托 别人 照 管 、 养 育 , 父母 则 奔 波 各地 谋 生 。"
300346,một thời làng phù lưu là làng có nhiều con nuô...,"我 村 曾是 北部 家里 儿女 认 干 亲 最多的 村庄 ""。"


In [13]:
df_test["vi"] = df_test["vi"].apply(remove_punc)
df_test["zh"] = df_test["zh"].apply(remove_punc)
df_test

,vi,zh
0,cô dung đã thiết kế và dành ra hai ngày để viế...,蓉 女士 设计 并 花了 两天 时间为 班 上的 00 名 学生 撰写 评论
1,nhân dân triều tiên thường yêu mến gọi ông là ...,"朝鲜 人 喜欢 叫他 白马 将军 , 为什么 呢 ?"
2,mỗi năm tôi chọn bốn sinh viên đến làm việc ch...,每年 我 会 挑选 0 名学生 为 我 工作
3,"theo chuyên gia wy ck off , nhà đ â ̀ u tư ban...","根据 威 科夫 的说法 , 投资者 最初 对 围绕 c ov id -00 疫苗 的消息 ..."
4,nhân viên ngân hàng kỹ năng siêu trộm : móc 0 ...,银行 员工 超级 小 偷 技巧 : 从 at m 上 钩 00 亿 太 容易 了
...,...,...
995,tôi không thể bảo vệ anh . vậy nên tôi muốn an...,我 不能 保护 你 所以 我要 你 离开 这里
996,"lượng rau củ quả , trái cây trung quốc hiện vẫ...","中国 蔬菜 和 水果 在 市场 上的 销售 量 仍然 不少 , 但 数量 已 大幅 下降 。"
997,ngành khách sạn khó thoát khỏi vòng lao đao tí...,"酒店 业 难 跳 出 周期 截至 0 月 , 胡志明市 旅游 局 报告 称 , 约 00-..."
998,"nhiều dự báo cho rằng , vàng sẽ còn tiếp tục t...","许多 预测 称 , 在中 美 冲突 高 涨 、 美国 大选 临近 、 新 冠 疫情 仍 不可..."


In [14]:
df_train['vi']   = df_train['vi'].astype(str)
df_train['zh']  = df_train['zh'].astype(str)

df_test['vi']   = df_test['vi'].astype(str)
df_test['zh']  = df_test['zh'].astype(str)

In [15]:
! pip install datasets transformers sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
from datasets import load_metric

metric = load_metric("sacrebleu")
metric

<ipython-input-16-dbdd67d4ba40>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


Metric(name: "sacrebleu", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions (`list` of `str`): list of translations to score. Each translation should be tokenized into a list of tokens.
    references (`list` of `list` of `str`): A list of lists of references. The contents of the first sub-list are the references for the first prediction, the contents of the second sub-list are for the second prediction, etc. Note that there must be the same number of references for each prediction (i.e. all sub-lists must be of the same length).
    smooth_method (`str`): The smoothing method to use, defaults to `'exp'`. Possible values are:
        - `'none'`: no smoothing
        - `'floor'`: increment zero counts
        - `'add-k'`: increment num/deno

In [17]:
language_codes = {
    'af': 'Afrikaans',
    'af-ZA': 'Afrikaans (South Africa)',
    'ar': 'Arabic',
    'ar-AE': 'Arabic (U.A.E.)',
    'ar-BH': 'Arabic (Bahrain)',
    'ar-DZ': 'Arabic (Algeria)',
    'ar-EG': 'Arabic (Egypt)',
    'ar-IQ': 'Arabic (Iraq)',
    'ar-JO': 'Arabic (Jordan)',
    'ar-KW': 'Arabic (Kuwait)',
    'ar-LB': 'Arabic (Lebanon)',
    'ar-LY': 'Arabic (Libya)',
    'ar-MA': 'Arabic (Morocco)',
    'ar-OM': 'Arabic (Oman)',
    'ar-QA': 'Arabic (Qatar)',
    'ar-SA': 'Arabic (Saudi Arabia)',
    'ar-SY': 'Arabic (Syria)',
    'ar-TN': 'Arabic (Tunisia)',
    'ar-YE': 'Arabic (Yemen)',
    'az': 'Azeri (Latin)',
    'az-AZ': 'Azeri (Latin) (Azerbaijan)',
    'az-AZ': 'Azeri (Cyrillic) (Azerbaijan)',
    'be': 'Belarusian',
    'be-BY': 'Belarusian (Belarus)',
    'bg': 'Bulgarian',
    'bg-BG': 'Bulgarian (Bulgaria)',
    'bs-BA': 'Bosnian (Bosnia and Herzegovina)',
    'ca': 'Catalan',
    'ca-ES': 'Catalan (Spain)',
    'cs': 'Czech',
    'cs-CZ': 'Czech (Czech Republic)',
    'cy': 'Welsh',
    'cy-GB': 'Welsh (United Kingdom)',
    'da': 'Danish',
    'da-DK': 'Danish (Denmark)',
    'de': 'German',
    'de-AT': 'German (Austria)',
    'de-CH': 'German (Switzerland)',
    'de-DE': 'German (Germany)',
    'de-LI': 'German (Liechtenstein)',
    'de-LU': 'German (Luxembourg)',
    'dv': 'Divehi',
    'dv-MV': 'Divehi (Maldives)',
    'el': 'Greek',
    'el-GR': 'Greek (Greece)',
    'en': 'English',
    'en-AU': 'English (Australia)',
    'en-BZ': 'English (Belize)',
    'en-CA': 'English (Canada)',
    'en-CB': 'English (Caribbean)',
    'en-GB': 'English (United Kingdom)',
    'en-IE': 'English (Ireland)',
    'en-JM': 'English (Jamaica)',
    'en-NZ': 'English (New Zealand)',
    'en-PH': 'English (Republic of the Philippines)',
    'en-TT': 'English (Trinidad and Tobago)',
    'en-US': 'English (United States)',
    'en-ZA': 'English (South Africa)',
    'en-ZW': 'English (Zimbabwe)',
    'eo': 'Esperanto',
    'es': 'Spanish',
    'es-AR': 'Spanish (Argentina)',
    'es-BO': 'Spanish (Bolivia)',
    'es-CL': 'Spanish (Chile)',
    'es-CO': 'Spanish (Colombia)',
    'es-CR': 'Spanish (Costa Rica)',
    'es-DO': 'Spanish (Dominican Republic)',
    'es-EC': 'Spanish (Ecuador)',
    'es-ES': 'Spanish (Castilian)',
    'es-ES': 'Spanish (Spain)',
    'es-GT': 'Spanish (Guatemala)',
    'es-HN': 'Spanish (Honduras)',
    'es-MX': 'Spanish (Mexico)',
    'es-NI': 'Spanish (Nicaragua)',
    'es-PA': 'Spanish (Panama)',
    'es-PE': 'Spanish (Peru)',
    'es-PR': 'Spanish (Puerto Rico)',
    'es-PY': 'Spanish (Paraguay)',
    'es-SV': 'Spanish (El Salvador)',
    'es-UY': 'Spanish (Uruguay)',
    'es-VE': 'Spanish (Venezuela)',
    'et': 'Estonian',
    'et-EE': 'Estonian (Estonia)',
    'eu': 'Basque',
    'eu-ES': 'Basque (Spain)',
    'fa': 'Farsi',
    'fa-IR': 'Farsi (Iran)',
    'fi': 'Finnish',
    'fi-FI': 'Finnish (Finland)',
    'fo': 'Faroese',
    'fo-FO': 'Faroese (Faroe Islands)',
    'fr': 'French',
    'fr-BE': 'French (Belgium)',
    'fr-CA': 'French (Canada)',
    'fr-CH': 'French (Switzerland)',
    'fr-FR': 'French (France)',
    'fr-LU': 'French (Luxembourg)',
    'fr-MC': 'French (Principality of Monaco)',
    'gl': 'Galician',
    'gl-ES': 'Galician (Spain)',
    'gu': 'Gujarati',
    'gu-IN': 'Gujarati (India)',
    'he': 'Hebrew',
    'he-IL': 'Hebrew (Israel)',
    'hi': 'Hindi',
    'hi-IN': 'Hindi (India)',
    'hr': 'Croatian',
    'hr-BA': 'Croatian (Bosnia and Herzegovina)',
    'hr-HR': 'Croatian (Croatia)',
    'hu': 'Hungarian',
    'hu-HU': 'Hungarian (Hungary)',
    'hy': 'Armenian',
    'hy-AM': 'Armenian (Armenia)',
    'id': 'Indonesian',
    'id-ID': 'Indonesian (Indonesia)',
    'is': 'Icelandic',
    'is-IS': 'Icelandic (Iceland)',
    'it': 'Italian',
    'it-CH': 'Italian (Switzerland)',
    'it-IT': 'Italian (Italy)',
    'ja': 'Japanese',
    'ja-JP': 'Japanese (Japan)',
    'ka': 'Georgian',
    'ka-GE': 'Georgian (Georgia)',
    'kk': 'Kazakh',
    'kk-KZ': 'Kazakh (Kazakhstan)',
    'kn': 'Kannada',
    'kn-IN': 'Kannada (India)',
    'ko': 'Korean',
    'ko-KR': 'Korean (Korea)',
    'kok': 'Konkani',
    'kok-IN': 'Konkani (India)',
    'ky': 'Kyrgyz',
    'ky-KG': 'Kyrgyz (Kyrgyzstan)',
    'lt': 'Lithuanian',
    'lt-LT': 'Lithuanian (Lithuania)',
    'lv': 'Latvian',
    'lv-LV': 'Latvian (Latvia)',
    'mi': 'Maori',
    'mi-NZ': 'Maori (New Zealand)',
    'mk': 'FYRO Macedonian',
    'mk-MK': 'FYRO Macedonian (Former Yugoslav Republic of Macedonia)',
    'mn': 'Mongolian',
    'mn-MN': 'Mongolian (Mongolia)',
    'mr': 'Marathi',
    'mr-IN': 'Marathi (India)',
    'ms': 'Malay',
    'ms-BN': 'Malay (Brunei Darussalam)',
    'ms-MY': 'Malay (Malaysia)',
    'mt': 'Maltese',
    'mt-MT': 'Maltese (Malta)',
    'nb': 'Norwegian (Bokmål)',
    'nb-NO': 'Norwegian (Bokmål) (Norway)',
    'nl': 'Dutch',
    'nl-BE': 'Dutch (Belgium)',
    'nl-NL': 'Dutch (Netherlands)',
    'nn-NO': 'Norwegian (Nynorsk) (Norway)',
    'ns': 'Northern Sotho',
    'ns-ZA': 'Northern Sotho (South Africa)',
    'pa': 'Punjabi',
    'pa-IN': 'Punjabi (India)',
    'pl': 'Polish',
    'pl-PL': 'Polish (Poland)',
    'ps': 'Pashto',
    'ps-AR': 'Pashto (Afghanistan)',
    'pt': 'Portuguese',
    'pt-BR': 'Portuguese (Brazil)',
    'pt-PT': 'Portuguese (Portugal)',
    'qu': 'Quechua',
    'qu-BO': 'Quechua (Bolivia)',
    'qu-EC': 'Quechua (Ecuador)',
    'qu-PE': 'Quechua (Peru)',
    'ro': 'Romanian',
    'ro-RO': 'Romanian (Romania)',
    'ru': 'Russian',
    'ru-RU': 'Russian (Russia)',
    'sa': 'Sanskrit',
    'sa-IN': 'Sanskrit (India)',
    'se': 'Sami (Northern)',
    'se-FI': 'Sami (Northern) (Finland)',
    'se-FI': 'Sami (Skolt) (Finland)',
    'se-FI': 'Sami (Inari) (Finland)',
    'se-NO': 'Sami (Northern) (Norway)',
    'se-NO': 'Sami (Lule) (Norway)',
    'se-NO': 'Sami (Southern) (Norway)',
    'se-SE': 'Sami (Northern) (Sweden)',
    'se-SE': 'Sami (Lule) (Sweden)',
    'se-SE': 'Sami (Southern) (Sweden)',
    'sk': 'Slovak',
    'sk-SK': 'Slovak (Slovakia)',
    'sl': 'Slovenian',
    'sl-SI': 'Slovenian (Slovenia)',
    'sq': 'Albanian',
    'sq-AL': 'Albanian (Albania)',
    'sr-BA': 'Serbian (Latin) (Bosnia and Herzegovina)',
    'sr-BA': 'Serbian (Cyrillic) (Bosnia and Herzegovina)',
    'sr-SP': 'Serbian (Latin) (Serbia and Montenegro)',
    'sr-SP': 'Serbian (Cyrillic) (Serbia and Montenegro)',
    'sv': 'Swedish',
    'sv-FI': 'Swedish (Finland)',
    'sv-SE': 'Swedish (Sweden)',
    'sw': 'Swahili',
    'sw-KE': 'Swahili (Kenya)',
    'syr': 'Syriac',
    'syr-SY': 'Syriac (Syria)',
    'ta': 'Tamil',
    'ta-IN': 'Tamil (India)',
    'te': 'Telugu',
    'te-IN': 'Telugu (India)',
    'th': 'Thai',
    'th-TH': 'Thai (Thailand)',
    'tl': 'Tagalog',
    'tl-PH': 'Tagalog (Philippines)',
    'tn': 'Tswana',
    'tn-ZA': 'Tswana (South Africa)',
    'tr': 'Turkish',
    'tr-TR': 'Turkish (Turkey)',
    'tt': 'Tatar',
    'tt-RU': 'Tatar (Russia)',
    'ts': 'Tsonga',
    'uk': 'Ukrainian',
    'uk-UA': 'Ukrainian (Ukraine)',
    'ur': 'Urdu',
    'ur-PK': 'Urdu (Islamic Republic of Pakistan)',
    'uz': 'Uzbek (Latin)',
    'uz-UZ': 'Uzbek (Latin) (Uzbekistan)',
    'uz-UZ': 'Uzbek (Cyrillic) (Uzbekistan)',
    'vi': 'Vietnamese',
    'vi-VN': 'Vietnamese (Viet Nam)',
    'xh': 'Xhosa',
    'xh-ZA': 'Xhosa (South Africa)',
    'zh': 'Chinese',
    'zh-CN': 'Chinese (S)',
    'zh-HK': 'Chinese (Hong Kong)',
    'zh-MO': 'Chinese (Macau)',
    'zh-SG': 'Chinese (Singapore)',
    'zh-TW': 'Chinese (T)',
    'zu': 'Zulu',
    'zu-ZA': 'Zulu (South Africa)'
}

In [18]:
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer


src_lang = "vi"
tgt_lang = "zh"
max_length = 128

model_name = "google/byt5-small"

def get_model(model_name):
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

  model_type = model.config.model_type


  if "mbart" in model_type:
      langs = "ar_AR,cs_CZ,de_DE,en_XX,es_XX,et_EE,fi_FI,fr_XX,gu_IN,hi_IN,it_IT,ja_XX,kk_KZ,ko_KR,lt_LT,lv_LV,my_MM,ne_NP,nl_XX,ro_RO,ru_RU,si_LK,tr_TR,vi_VN,zh_CN"
      langs = langs.split(",")

      for lang in langs:
          if src_lang in lang:
              tokenizer.src_lang = lang

          if tgt_lang in lang:
              tokenizer.tgt_lang = lang

  if "t5" in model_type:
      prefix = f"translate {language_codes.get(src_lang, src_lang)} to {language_codes.get(tgt_lang, tgt_lang)} : "
  else:
      prefix = ""

  print(f"MODE TYPE\t:\t{model_type}")

  if "mbart" in model_type:
      print(f"SRC_LANG \t:\t{tokenizer.src_lang}")
      print(f"TGT_LANG \t:\t{tokenizer.tgt_lang}")

  if prefix:
      print(f"PREFIX   \t:\t{prefix[:-3]}")
  else:
      print(f"PREFIX   \t:\tNone")

  return model, tokenizer, model_type, prefix

In [19]:
def preprocess_function(tokenizer, src, tgt):
    inputs = prefix + src
    targets = tgt
    model_inputs = tokenizer(inputs, max_length=max_length, truncation=True)

    labels = tokenizer(text_target = targets, max_length=max_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [20]:
!pip install accelerate -U
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__ (self, tokenizer, prefix, X_data, y_data):
        self.tokenizer = tokenizer
        self.X_data = X_data
        self.y_data = y_data
        self.prefix = prefix
    def __len__ (self):
        return len(self.X_data)

    def __getitem__(self, idx):
        inputs = self.prefix + self.X_data[idx]
        targets = self.y_data[idx]

        inputs = self.tokenizer(inputs, max_length=max_length, truncation=True, return_tensors="pt", padding="max_length",)

        labels = self.tokenizer(text_target = targets, max_length=max_length, truncation=True, return_tensors="pt", padding="max_length",)

        return {
            'input_ids' : inputs.input_ids.squeeze(),
            'attention_mask': inputs.attention_mask.squeeze(),
            'labels': labels.input_ids.squeeze(),
        }

In [22]:
import wandb

# bda323eb3484c7dab0e1d5fe7d6812d8f4e79f22

wandb.login()

# set the wandb project where this run will be logged
os.environ["WANDB_PROJECT"]="UIT_NCKH_MACHINE_TRANSLATION"

# save your trained model checkpoint to wandb
os.environ["WANDB_LOG_MODEL"]="end"

wandb: Currently logged in as: it-dainb. Use `wandb login --relogin` to force relogin


In [23]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}

    return result

In [ ]:
free_gpu()

model, tokenizer, model_type, prefix = get_model(model_name)

from datetime import datetime

now = datetime.now()
dt_string = now.strftime("%d-%m-%Y_%H-%M-%S")

max_train = -1
if max_train > 0:
    train_dataset = CustomDataset(tokenizer, prefix, df_train[src_lang][:max_train], df_train[tgt_lang][:max_train])
else:
    train_dataset = CustomDataset(tokenizer, prefix, df_train[src_lang], df_train[tgt_lang])

test_dataset  = CustomDataset(tokenizer, prefix, df_test[src_lang], df_test[tgt_lang])

epochs = 1
learning_rate = 1e-3
weight_decay = 0.01
logging_steps = 100
batch_size = 128

model_name = model_name.replace("/", "_")
training_args = Seq2SeqTrainingArguments(
    output_dir = f'/content/gdrive/MyDrive/Machine_Translation/models/{model_type}',
    overwrite_output_dir = True,

    learning_rate = learning_rate,
    num_train_epochs = epochs,
    weight_decay = weight_decay,
    optim = "adamw_torch",

    # evaluation_strategy= 'epoch',
    save_strategy = 'no',
    # save_steps = logging_steps * 50,
    # eval_steps = logging_steps * 10,

    # load_best_model_at_end = True,
    # auto_find_batch_size = True,
    per_device_train_batch_size = batch_size,


    fp16=False,
    # fp16_full_eval=True,
    # jit_mode_eval=True,
    gradient_accumulation_steps = 1,
    gradient_checkpointing=True,

    logging_strategy="steps",
    logging_steps = logging_steps,

    seed = seed,

    report_to = "wandb",
    run_name = model_type + f"_{model_name}_{dt_string}"
)

trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    # eval_dataset = test_dataset,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics,
    data_collator = lambda data : {
        'input_ids' : torch.stack([item['input_ids'] for item in data]),
        'attention_mask' : torch.stack([item['attention_mask'] for item in data]),
        'labels' : torch.stack([item['labels'] for item in data]),
    }
)

trainer.train()

wandb.finish()

free_gpu()

test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

model.eval()

pred = []
label = []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model.generate(input_ids, attention_mask=attention_mask, max_length=max_length)

        labels = batch['labels'].to(device)

        decoded_preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        # Some simple post-processing
        decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
        pred += decoded_preds
        label += decoded_labels



result = metric.compute(predictions=pred, references=label)
result = {"bleu": result["score"]}
print(result)


model.train()

MODE TYPE	:	t5
PREFIX   	:	translate Vietnamese to Chinese


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
100,3.067100
200,0.869300
300,0.830800
400,0.807500
500,0.790200
600,0.777800
700,0.772300
800,0.762300
900,0.756800
1000,0.749500


Step,Training Loss
100,3.067100
200,0.869300
300,0.830800
400,0.807500
500,0.790200
600,0.777800
700,0.772300
800,0.762300
900,0.756800
1000,0.749500


In [ ]:
pred

In [ ]:
label